# AutoGen Verktygsanvändningsexempel


## Importera de nödvändiga paketen


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Skapa klienten

I det här exemplet kommer vi att använda [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) för att få tillgång till LLM.

`model` är definierad som `gpt-4o-mini`. Prova att byta modellen till en annan modell som finns tillgänglig på GitHub Models-marknadsplatsen för att se olika resultat.

Som ett snabbt test kommer vi att köra en enkel fråga - `What is the capital of France`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definiera funktionerna

I det här exemplet kommer vi att ge agenten tillgång till ett verktyg som är en funktion med en lista över tillgängliga semesterdestinationer och deras tillgänglighet.

Du kan tänka dig att detta skulle vara ett scenario där en reseagent till exempel kan ha tillgång till en resedatabas.

När du går igenom detta exempel, känn dig fri att försöka definiera nya funktioner och verktyg som agenten kan använda.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Definiera verktyget Funktion 
För att agenten ska kunna använda `vacation_destinations` som ett `FunctionTool`, måste vi definiera det som ett sådant.

Vi kommer också att ge en beskrivning av verktyget, vilket hjälper agenten att förstå vad verktyget används till i förhållande till den uppgift användaren har begärt.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Definiera agenten

Nu kan vi skapa agenten i koden nedan. Vi definierar `system_message` för att ge agenten instruktioner om hur den ska hjälpa användare att hitta semesterresmål.

Vi ställer också in parametern `reflect_on_tool_use` till true. Detta gör det möjligt att använda LLM för att ta svaret från verktygsanropet och skicka svaret med naturligt språk.

Du kan ställa in parametern till false för att se skillnaden.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Köra agenten

Nu kan vi köra agenten med det första användarmeddelandet som ber om att göra en resa till Tokyo.

Du kan ändra denna stadsdestination för att se hur agenten svarar på tillgängligheten för staden.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör det noteras att automatiserade översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
